In [2]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import warnings
warnings.filterwarnings("ignore")

from sklearn import tree
from sklearn.tree import _tree
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pylab as plt

from itertools import combinations
#del list

import math

In [256]:
Indt_core_train = pd.read_csv('Indt_CORE_train_2019-03-07.csv')
Indt_core_test = pd.read_csv('Indt_CORE_test_2019-03-07.csv')

In [257]:
# 魔蝎蜂巢合并 2018.08.20 仅训练集
row = Indt_core_train.shape[0]
mean_var44_train = Indt_core_train[Indt_core_train['var44']>0]['var44'].mean()
mean_var46_train = Indt_core_train[Indt_core_train['var46']>0]['var46'].mean()
mean_var47_train = Indt_core_train[Indt_core_train['var47']>0]['var47'].mean()
for i in range(row):
    if (Indt_core_train.ix[i,'var44'] == -3) or (Indt_core_train.ix[i,'var44'] == -2): #魔蝎_入网时长
        Indt_core_train.ix[i,'var44'] = mean_var44_train #Indt_core_train.ix[i,'var51']
    if (Indt_core_train.ix[i,'var46'] == -3) or (Indt_core_train.ix[i,'var46'] == -2): #魔蝎_夜间活动比例
        Indt_core_train.ix[i,'var46'] = mean_var46_train #Indt_core_train.ix[i,'var50']
    if (Indt_core_train.ix[i,'var47'] == -3) or (Indt_core_train.ix[i,'var47'] == -2): #魔蝎_电话互通数量
        Indt_core_train.ix[i,'var47'] = mean_var47_train #Indt_core_train.ix[i,'var48']   

In [258]:
mean_var8_train = Indt_core_train[Indt_core_train['var8']<999]['var8'].mean()
mean_var8_test = Indt_core_test[Indt_core_test['var8']<999]['var8'].mean()        
Indt_core_train['var8'] = Indt_core_train['var8'].map(lambda x: mean_var8_train if x==9999 else x) #算话_30天内单位信息被使用人数
Indt_core_test['var8'] = Indt_core_test['var8'].map(lambda x: mean_var8_test if x==9999 else x) #算话_30天内单位信息被使用人数

mean_var7_train = Indt_core_train[Indt_core_train['var7']<999]['var7'].mean()
mean_var7_test = Indt_core_test[Indt_core_test['var7']<999]['var7'].mean()        
Indt_core_train['var7'] = Indt_core_train['var7'].map(lambda x: mean_var7_train if x==9999 else x) #算话_30天内单位信息被使用人数
Indt_core_test['var7'] = Indt_core_test['var7'].map(lambda x: mean_var7_test if x==9999 else x) #算话_30天内单位信息被使用人数

# var27-var42 删除;  var48-var54 删除;  var1-var13 删除
col = []
for i in range(1,55):
    if i<=13 or (i>=27 and i<43) or (i>=48 and i<55):
        col.append("var" + str(i))
        col.append("var" + str(i) + "_cnt")
Indt_core_train.drop(columns = col,inplace=True)
Indt_core_test.drop(columns = col,inplace=True)

# 删除缺失值>10%的列
cnt = []
for i in Indt_core_train.columns:
    if "_cnt" in i:
        cnt.append(i)
cnt.remove('score_v1_cnt')
name = Indt_core_train[cnt].columns
missing = []
for i in name:
    if Indt_core_train.ix[:,i].mean() < 0.9:
        missing.append(i[0:5])
Indt_core_train.drop(columns = missing,inplace=True)
Indt_core_test.drop(columns = missing,inplace=True)

In [259]:
def var(df):
    name = df.columns
    
    for i in name:
        if "_cnt" in i:
            df = df.drop(columns = [i])

    df = df.drop(columns = ['LOAN_NO', 'CERT_NO', 'PHONE_NO', 'CUST_NAME', 'REG_DATE',
       'CHANNEL_NAME', 'APPLY_DATE', 'PROD_NO', 'PAY_AMT', 'LOAN_DATE','LOAN_AMT', 
       'ACT_LOAN_AMT', 'DUE_DATE', 'ACT_REPAY_DATE', 'REPAY_STAT','OVDU_DAYS',
       'LOAN_NO.1', 'score_v1','layer','cert_no','nDay_Next_Odue'])
    return df

Indt_core_train = var(Indt_core_train)
Indt_core_test = var(Indt_core_test)

In [260]:
def value_count(df):
    name=df.columns
    lenth=[]
    names=[]
    for i in name:
        lenth.append(len(df[i].value_counts()))
        names.append(i)
    lenth=pd.DataFrame(lenth)
    names=pd.DataFrame(names)
    valuecounts=pd.concat([names,lenth],axis=1)
    valuecounts.columns=["name","vary_num"]
    return valuecounts

# 删除只有一值的数据
df = value_count(Indt_core_train)
list = df[df['vary_num'] == 1]['name'].tolist()
Indt_core_train.drop(columns = list,inplace=True)
Indt_core_test.drop(columns = list,inplace=True)

In [261]:
Indt_core_train['jyzx_min_Inquiry_date_gap'] = Indt_core_train['jyzx_min_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)
Indt_core_test['jyzx_min_Inquiry_date_gap'] = Indt_core_test['jyzx_min_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)
Indt_core_train['jyzx_max_Inquiry_date_gap'] = Indt_core_train['jyzx_max_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)
Indt_core_test['jyzx_max_Inquiry_date_gap'] = Indt_core_test['jyzx_max_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)
Indt_core_train['af_min_Inquiry_date_gap'] = Indt_core_train['af_min_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)
Indt_core_test['af_min_Inquiry_date_gap'] = Indt_core_test['af_min_Inquiry_date_gap'].map(lambda x: 0 if (x<0) else x)

In [262]:
# 客户婚姻状况分类
# 84000001 未婚;  84000002 已婚有子女;  84000003 已婚无子女;  84000004 离异;  84000005 丧偶
Indt_core_train['ind_married'] = Indt_core_train['var61'].map(lambda x: 1 if (x==84000001 or x==84000004) else 0)
Indt_core_train['ind_divorced'] = Indt_core_train['var61'].map(lambda x: 0 if (x==84000004) else 1)
Indt_core_train['ind_parent'] = Indt_core_train['var61'].map(lambda x: 0 if (x==84000002) else 1)
Indt_core_test['ind_married'] = Indt_core_test['var61'].map(lambda x: 1 if (x==84000001 or x==84000004) else 0)
Indt_core_test['ind_divorced'] = Indt_core_test['var61'].map(lambda x: 0 if (x==84000004) else 1)
Indt_core_test['ind_parent'] = Indt_core_test['var61'].map(lambda x: 0 if (x==84000002) else 1)

In [263]:
train = Indt_core_train
test = Indt_core_test
X_train = Indt_core_train.drop("target",axis=1)
y_train = Indt_core_train["target"]
X_test = Indt_core_test.drop("target",axis=1)
y_test = Indt_core_test["target"]

In [264]:
X_train.shape

(11042, 654)

In [265]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.99 * (1 - .99))) 

s = sel.fit_transform(X_train)

indices = sel.get_support(False)
feat_labels = X_train.columns
features = feat_labels[indices]
X_train = X_train.loc[:,features]
X_test = X_test.loc[:,features]

In [266]:
from sklearn.feature_selection import SelectKBest,chi2,SelectFpr
from sklearn.feature_selection import GenericUnivariateSelect

sp = SelectFpr(chi2, alpha=0.05)
s = sp.fit_transform(X_train, y_train)

indices = sp.get_support(False)
feat_labels = X_train.columns
features = feat_labels[indices]
X_train = X_train.loc[:,features]
X_test = X_test.loc[:,features]

#### PSI

In [267]:
ps = pd.DataFrame()
drop_cols = []
for i in X_train.columns:
    # 划分测试集
    cols_train = X_train.sort_values(i, inplace=False)[i]
    cats_train = pd.qcut(cols_train, 10, duplicates="drop")
    ####cats_train = pd.cut(cols_train, 10)
    ps["train_pct"] = cols_train.groupby(cats_train).count()/ X_train.shape[0]
    
    # 划分训练集
    bin = [-0.001]
    for j in cats_train.unique(): 
        bin.append(j.right)
    

    cols_test = X_test.sort_values(i, inplace=False)[i]
    cats_test = pd.cut(cols_test, bin)
    ps["test_pct"] = pd.cut(cols_test, bin).groupby(cats_test).count()/ X_test.shape[0]
    
    # 计算PSI    
    psi = ((ps["train_pct"]-ps["test_pct"])*((ps["train_pct"]/ps["test_pct"]).apply(lambda x: math.log(x)))).sum()*100
    if psi>=25: 
        drop_cols.append(i) 
        print(i, psi, bin)
    
    ps = pd.DataFrame()

jyzx_max_Inquiry_360d 27.610110791074593 [-0.001, 500.0, 1500.0, 2500.0, 3500.0, 5000.0, 15000.0, 310000.0]
jyzx_max_Repay_Unclear_180d 25.009026012543828 [-0.001, 500.0, 1500.0, 2500.0, 5000.0, 210000.0]
jyzx_max_Repay_Unclear_360d 26.827678382777957 [-0.001, 500.0, 1500.0, 2500.0, 5000.0, 15000.0, 310000.0]
jyzx_max_Repay_Unclear_90d 27.521753531388683 [-0.001, 500.0, 1500.0, 2500.0, 190000.0]
jyzx_max_Repay_Unknown_all 47.51495279515547 [-0.001, 1500.0, 310000.0]
jyzx_max_Type_Personal_360d 26.938866716361776 [-0.001, 500.0, 1500.0, 2500.0, 3500.0, 5000.0, 15000.0, 210000.0]
jyzx_max_Type_PersonalCredit_360d 27.023051346357384 [-0.001, 500.0, 1500.0, 2500.0, 3500.0, 5000.0, 15000.0, 210000.0]
jyzx_sum_Repay_Unclear_180d 28.360089298524095 [-0.001, 500.0, 1500.0, 3000.0, 7000.0, 450000.0]
jyzx_sum_Repay_Unclear_360d 29.63477568870524 [-0.001, 500.0, 1500.0, 3000.0, 5000.0, 15000.0, 452000.0]
jyzx_sum_Repay_Unclear_90d 26.592275928843456 [-0.001, 850.0, 2500.0, 3500.0, 395500.0]
jyzx_

In [269]:
X_train = X_train.drop(columns = drop_cols)
X_test = X_test.drop(columns = drop_cols)

In [270]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
sfm = SelectFromModel(LogisticRegression(penalty="l1", C=0.1))
s = sfm.fit_transform(X_train, y_train)

indices = sfm.get_support(False)
feat_labels = X_train.columns
features = feat_labels[indices]
X_train = X_train.loc[:,features]
X_test = X_test.loc[:,features]

In [271]:
# 计算斯皮尔曼相关系数（矩阵）
X_train_corr = X_train.corr(method='spearman')

# 将对角线变为0
mask = np.ones(X_train_corr.columns.size) - np.eye(X_train_corr.columns.size)
X_train_corr = mask * X_train_corr

drops = []
for col in X_train_corr.columns.values:
    # if we've already determined to drop the current variable, continue
    if np.in1d([col],drops):
        continue

    # 找出高相关的变量
    corr = X_train_corr[abs(X_train_corr[col]) > 0.85].index
    drops = np.union1d(drops, corr)

print("nDropping", drops.shape[0], "highly correlated features...n", drops)

nDropping 233 highly correlated features...n ['af_max_Inquiry_Approved_90d' 'af_max_Inquiry_Checking_360d'
 'af_max_Inquiry_Checking_90d' 'af_max_Inquiry_Checking_all'
 'af_max_Repay_M1_360d' 'af_max_Repay_M1_all' 'af_max_Repay_M1plus_30d'
 'af_max_Repay_M1plus_360d' 'af_max_Repay_M1plus_90d'
 'af_max_Repay_M1plus_all' 'af_max_Repay_Normal_30d'
 'af_max_Repay_Normal_360d' 'af_max_Repay_Normal_90d'
 'af_max_Repay_Odue_180d' 'af_max_Repay_Odue_30d' 'af_max_Repay_Odue_360d'
 'af_max_Repay_Odue_90d' 'af_max_Repay_Odue_all' 'af_max_Type_Assure_all'
 'af_max_Type_Credit_30d' 'af_max_Type_Credit_360d'
 'af_max_Type_Credit_90d' 'af_mean_Period_all' 'af_min_Odue_date_gap'
 'af_n_Inquiry_90d' 'af_n_Inquiry_Approved_90d'
 'af_n_Inquiry_Pass_rate_30d' 'af_n_Inquiry_Rejected_30d'
 'af_n_Inquiry_Rejected_360d' 'af_n_Inquiry_Rejected_all'
 'af_n_Inquiry_all' 'af_n_Repay_Clear_180d' 'af_n_Repay_Clear_90d'
 'af_n_Type_Credit_360d' 'af_n_Type_Credit_90d' 'af_n_Type_Credit_all'
 'af_sum_Inquiry_Aborted_1

In [272]:
X_train = X_train.drop(columns = drops)
X_test = X_test.drop(columns = drops)

In [274]:
X_train.shape

(11042, 109)

#### 随机森林

In [185]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

forest = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1, max_depth=7, min_samples_split=130, max_features=3)

param_dist = {
        #'max_depth':range(2,10,1), #overfitting
        #'min_samples_split':range(110, 201, 10),
        'max_features':range(5,11,1),
        #'min_samples_leaf':range(10, 60, 10) #overfitting 
        }

#n_iter=300，训练300次，数值越大，获得的参数精度越大，但是搜索时间越长
grid = GridSearchCV(forest, param_dist, cv=3, scoring='roc_auc')
grid.fit(X_train, y_train)

best_estimator = grid.best_estimator_ #返回最优的训练器
print(best_estimator)
print(grid.best_score_)
display(pd.DataFrame(grid.cv_results_).T)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features=8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=130,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
0.6546133583103999


,0,1,2,3,4,5
mean_fit_time,0.422297,0.478912,0.482749,0.490574,0.498346,0.606369
std_fit_time,0.020734,0.0503743,0.0601478,0.0417837,0.0136533,0.0771715
mean_score_time,0.113294,0.116675,0.11619,0.11816,0.118361,0.120605
std_score_time,0.00182439,0.00537657,0.00345257,0.00270582,0.00590169,0.00343426
param_max_features,5,6,7,8,9,10
params,{'max_features': 5},{'max_features': 6},{'max_features': 7},{'max_features': 8},{'max_features': 9},{'max_features': 10}
split0_test_score,0.655811,0.654495,0.656008,0.661045,0.657149,0.659229
split1_test_score,0.639131,0.634672,0.646885,0.64364,0.639162,0.640976
split2_test_score,0.654743,0.647977,0.648833,0.659156,0.654771,0.658916
mean_test_score,0.649895,0.645715,0.650576,0.654613,0.65036,0.65304


In [275]:
forest = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1, max_depth=7, min_samples_split=130, max_features=3)
forest.fit(X_train, y_train)

importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

feat_labels = X_train.columns
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) af_n_Inquiry_Rejected_180d     0.059759
 2) af_n_Query_LoanCheck_all       0.058056
 3) af_max_Odue_date_gap           0.045740
 4) var44                          0.028343
 5) af_max_Repay_M1_90d            0.028081
 6) var55                          0.024325
 7) af_sum_Inquiry_Rejected_90d    0.022565
 8) jyzx_n_Inquiry_10d             0.019688
 9) af_max_Inquiry_Rejected_30d    0.019415
10) af_max_Inquiry_date_gap        0.019092
11) af_sum_Inquiry_Pass_rate_all   0.017758
12) af_sum_Type_Credit_30d         0.017756
13) af_max_Repay_Clear_360d        0.016990
14) af_n_Inquiry_360d              0.016916
15) jyzx_max_Inquiry_Approved_360d 0.016728
16) af_n_Inquiry_Approved_all      0.015994
17) jyzx_max_Inquiry_180d          0.015698
18) var24                          0.015650
19) af_n_Query_P2P_all             0.015553
20) jyzx_n_Repay_Unclear_90d       0.014968
21) jyzx_n_Repay_Unclear_30d       0.014547
22) af_min_Inquiry_date_gap        0.014302
23) af_max_Repay_Normal_all     

In [284]:
indices_ = indices[0:32]
X_train_model = X_train.iloc[:,indices_]
X_test_model = X_test.iloc[:,indices_]

In [285]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression()
logistic_model.fit(X_train_model, y_train)

from sklearn.metrics import roc_curve

predict_train = logistic_model.predict_proba(X_train_model)
fpr,tpr,thresholds = roc_curve(y_train,predict_train[:,1])
print('KS-train:',max(tpr-fpr))

predict_test = logistic_model.predict_proba(X_test_model)
fpr,tpr,thresholds = roc_curve(y_test,predict_test[:,1])
print('KS-test:',max(tpr-fpr))

KS-train: 0.18483955763412868
KS-test: 0.18904236872718228


In [287]:
X_train_model.columns

Index(['af_n_Inquiry_Rejected_180d', 'af_n_Query_LoanCheck_all',
       'af_max_Odue_date_gap', 'var44', 'af_max_Repay_M1_90d', 'var55',
       'af_sum_Inquiry_Rejected_90d', 'jyzx_n_Inquiry_10d',
       'af_max_Inquiry_Rejected_30d', 'af_max_Inquiry_date_gap',
       'af_sum_Inquiry_Pass_rate_all', 'af_sum_Type_Credit_30d',
       'af_max_Repay_Clear_360d', 'af_n_Inquiry_360d',
       'jyzx_max_Inquiry_Approved_360d', 'af_n_Inquiry_Approved_all',
       'jyzx_max_Inquiry_180d', 'var24', 'af_n_Query_P2P_all',
       'jyzx_n_Repay_Unclear_90d', 'jyzx_n_Repay_Unclear_30d',
       'af_min_Inquiry_date_gap', 'af_max_Repay_Normal_all', 'var20', 'var59',
       'jyzx_max_arrears_360d', 'jyzx_n_Inquiry_Approved_all',
       'af_max_Inquiry_Approved_360d', 'af_max_Repay_Clear_all',
       'jyzx_min_Inquiry_date_gap', 'var47', 'var18'],
      dtype='object')

#### XGBoost

In [247]:
# 训练模型
model = xgb.XGBClassifier(max_depth=3, learning_rate=0.1, silent=True, subsample=0.8, objective='binary:logistic')
model.fit(X_train, y_train)

# 显示重要特征
#plot_importance(model)
#plt.show()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [248]:
from sklearn.ensemble import RandomForestClassifier

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

feat_labels = X_train.columns
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) af_max_Odue_date_gap           0.032082
 2) af_max_Repay_Clear_360d        0.027610
 3) var42                          0.026834
 4) jyzx_n_Inquiry_10d             0.026491
 5) jyzx_max_Repay_Clear_360d      0.026046
 6) af_max_Repay_Clear_90d         0.025712
 7) af_max_Repay_M1_90d            0.025627
 8) jyzx_max_Inquiry_360d          0.024871
 9) af_n_Query_LoanCheck_all       0.023083
10) jyzx_n_Inquiry_Checking_30d    0.022243
11) af_max_Repay_Normal_360d       0.021378
12) jyzx_max_Inquiry_all           0.021337
13) af_max_Repay_Clear_30d         0.021318
14) af_max_Inquiry_Rejected_30d    0.020580
15) af_n_Inquiry_Rejected_360d     0.020357
16) af_max_Inquiry_Approved_30d    0.020167
17) jyzx_n_Inquiry_Rejected_all    0.019310
18) af_max_Inquiry_Checking_180d   0.019162
19) var20                          0.019160
20) af_max_Inquiry_date_gap        0.018892
21) jyzx_min_Inquiry_date_gap      0.018271
22) jyzx_n_Repay_Paid_all          0.018189
23) var17                       

In [262]:
indices_ = indices[0:23]
X_train_model = X_train.iloc[:,indices_]
X_test_model = X_test.iloc[:,indices_]

#### 网格搜索

In [246]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
model = xgb.XGBClassifier(objective='binary:logistic')
#learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
#param_grid = dict(learning_rate=learning_rate)
#max_depth = [6, 5, 4, 3]
#param_grid = dict(max_depth=max_depth)
## Best: 0.871428 using {'max_depth': 3}
subsample = [1, 0.9, 0.8, 0.7]
param_grid = dict(subsample=subsample)
## Best: 0.871596 using {'subsample': 0.8}
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.871596 using {'subsample': 0.8}


In [371]:

#分类器使用 xgboost
clf1 = xgb.XGBClassifier(objective='binary:logistic')
 
#设定网格搜索的xgboost参数搜索范围，值搜索XGBoost的主要6个参数
param_dist = {
        #'n_estimators':range(80,200,4), #speed
        'max_depth':range(2,10,1), #overfitting
        'learning_rate':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3], #overfitting
        #'subsample':np.linspace(0.7,0.9,20), #speed
        #'colsample_bytree':np.linspace(0.5,0.98,10), #speed
        'min_child_weight':range(1,9,1) #overfitting
        }
 

#param_dist字典类型，放入参数搜索范围
#n_iter=300，训练300次，数值越大，获得的参数精度越大，但是搜索时间越长
grid = GridSearchCV(clf1, param_dist, cv=3, scoring='roc_auc')
 
#在训练集上训练
grid.fit(X_train, y_train)
#返回最优的训练器
best_estimator = grid.best_estimator_
print(best_estimator)
#输出最优训练器的精度
print(grid.best_score_)


KeyboardInterrupt: 

#### 枚举

In [305]:
cols = ['af_n_Inquiry_Rejected_180d', 'af_n_Query_LoanCheck_all',
       'af_max_Odue_date_gap', 'var44', 'af_max_Repay_M1_90d', 'var55',
       'af_sum_Inquiry_Rejected_90d', 'jyzx_n_Inquiry_10d',
       'af_max_Inquiry_Rejected_30d', 'af_max_Inquiry_date_gap',
       'af_sum_Inquiry_Pass_rate_all', 'af_sum_Type_Credit_30d',
       'af_max_Repay_Clear_360d', 'af_n_Inquiry_360d',
       'af_max_Repay_Normal_all', 'var20', 'jyzx_max_arrears_360d',
       'af_max_Inquiry_Approved_360d']

for var in combinations(cols, 15):
    li = list(var)
    X_train_model_3 = X_train_model[li]
    X_test_model_3 = X_test_model[li]
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train_model_3, y_train)
    
    predict_train = logistic_model.predict_proba(X_train_model_3)
    fpr,tpr,thresholds = roc_curve(y_train,predict_train[:,1])
    KS_train = max(tpr-fpr)

    if KS_train>0.2:
        predict_test = logistic_model.predict_proba(X_test_model_3)
        fpr,tpr,thresholds = roc_curve(y_test,predict_test[:,1])
        KS_test = max(tpr-fpr)

        if KS_test>0.2:
            print(X_train_model_3.columns,"KS_train:",KS_train,"KS_test",KS_test)
    
    del li

Index(['af_n_Inquiry_Rejected_180d', 'af_n_Query_LoanCheck_all',
       'af_max_Odue_date_gap', 'var44', 'af_max_Repay_M1_90d', 'var55',
       'af_sum_Inquiry_Rejected_90d', 'jyzx_n_Inquiry_10d',
       'af_max_Inquiry_Rejected_30d', 'af_sum_Inquiry_Pass_rate_all',
       'af_sum_Type_Credit_30d', 'af_n_Inquiry_360d',
       'af_max_Repay_Normal_all', 'jyzx_max_arrears_360d',
       'af_max_Inquiry_Approved_360d'],
      dtype='object') KS_train: 0.205054951410443 KS_test 0.20018063146222875
Index(['af_n_Inquiry_Rejected_180d', 'af_n_Query_LoanCheck_all',
       'af_max_Odue_date_gap', 'var44', 'af_max_Repay_M1_90d', 'var55',
       'af_sum_Inquiry_Rejected_90d', 'af_max_Inquiry_Rejected_30d',
       'af_max_Inquiry_date_gap', 'af_sum_Inquiry_Pass_rate_all',
       'af_max_Repay_Clear_360d', 'af_n_Inquiry_360d', 'var20',
       'jyzx_max_arrears_360d', 'af_max_Inquiry_Approved_360d'],
      dtype='object') KS_train: 0.20001623340238894 KS_test 0.2026576074541776
Index(['af_n_Inquiry_R

KeyboardInterrupt: 

In [ ]:
Index(['af_n_Inquiry_Rejected_180d', 'af_n_Query_LoanCheck_all',
       'af_max_Odue_date_gap', 'var44', 'af_max_Repay_M1_90d', 'var55',
       'af_sum_Inquiry_Rejected_90d', 'jyzx_n_Inquiry_10d',
       'af_max_Inquiry_Rejected_30d', 'af_max_Inquiry_date_gap',
       'af_sum_Inquiry_Pass_rate_all', 'af_sum_Type_Credit_30d',
       'af_max_Repay_Clear_360d', 'af_n_Inquiry_360d',
       'jyzx_max_Inquiry_Approved_360d', 'jyzx_max_Inquiry_180d', 'var20',
       'jyzx_min_Inquiry_date_gap'],
      dtype='object') KS_train: 0.21452031179727876 KS_test 0.2152416535176392
Index(['af_n_Inquiry_Rejected_180d', 'af_n_Query_LoanCheck_all',
       'af_max_Odue_date_gap', 'var44', 'af_max_Repay_M1_90d', 'var55',
       'af_sum_Inquiry_Rejected_90d', 'jyzx_n_Inquiry_10d',
       'af_max_Inquiry_Rejected_30d', 'af_max_Inquiry_date_gap',
       'af_sum_Inquiry_Pass_rate_all', 'af_sum_Type_Credit_30d',
       'af_max_Repay_Clear_360d', 'af_n_Inquiry_360d',
       'af_n_Inquiry_Approved_all', 'jyzx_max_Inquiry_180d',
       'af_n_Query_P2P_all', 'af_max_Inquiry_Approved_360d'],
      dtype='object') KS_train: 0.21325677898328704 KS_test 0.2156184432689579
Index(['af_n_Inquiry_Rejected_180d', 'af_n_Query_LoanCheck_all',
       'af_max_Odue_date_gap', 'var44', 'af_max_Repay_M1_90d', 'var55',
       'af_sum_Inquiry_Rejected_90d', 'jyzx_n_Inquiry_10d',
       'af_max_Inquiry_Rejected_30d', 'af_max_Inquiry_date_gap',
       'af_sum_Inquiry_Pass_rate_all', 'af_sum_Type_Credit_30d',
       'af_max_Repay_Clear_360d', 'af_n_Inquiry_360d',
       'af_max_Repay_Normal_all', 'var20', 'jyzx_max_arrears_360d',
       'af_max_Inquiry_Approved_360d'],
      dtype='object') KS_train: 0.217460212079221 KS_test 0.2112782480536594

In [314]:
X_train_model = X_train[['af_n_Inquiry_Rejected_180d', 'af_n_Query_LoanCheck_all',
       'af_max_Odue_date_gap', 'var44', 'af_max_Repay_M1_90d', 'var55',
       'af_sum_Inquiry_Rejected_90d', 'jyzx_n_Inquiry_10d',
       'af_max_Inquiry_Rejected_30d', 'af_max_Inquiry_date_gap',
       'af_sum_Inquiry_Pass_rate_all', 'af_sum_Type_Credit_30d',
       'af_max_Repay_Clear_360d', 'af_n_Inquiry_360d',
       'af_max_Repay_Normal_all', 'var20', 'jyzx_max_arrears_360d',
       'af_max_Inquiry_Approved_360d'
       ]]
X_test_model = X_test[['af_n_Inquiry_Rejected_180d', 'af_n_Query_LoanCheck_all',
       'af_max_Odue_date_gap', 'var44', 'af_max_Repay_M1_90d', 'var55',
       'af_sum_Inquiry_Rejected_90d', 'jyzx_n_Inquiry_10d',
       'af_max_Inquiry_Rejected_30d', 'af_max_Inquiry_date_gap',
       'af_sum_Inquiry_Pass_rate_all', 'af_sum_Type_Credit_30d',
       'af_max_Repay_Clear_360d', 'af_n_Inquiry_360d',
       'af_max_Repay_Normal_all', 'var20', 'jyzx_max_arrears_360d',
       'af_max_Inquiry_Approved_360d'
       ]]

In [315]:
for i in X_test_model.columns:
    print(i)

af_n_Inquiry_Rejected_180d
af_n_Query_LoanCheck_all
af_max_Odue_date_gap
var44
af_max_Repay_M1_90d
var55
af_sum_Inquiry_Rejected_90d
jyzx_n_Inquiry_10d
af_max_Inquiry_Rejected_30d
af_max_Inquiry_date_gap
af_sum_Inquiry_Pass_rate_all
af_sum_Type_Credit_30d
af_max_Repay_Clear_360d
af_n_Inquiry_360d
af_max_Repay_Normal_all
var20
jyzx_max_arrears_360d
af_max_Inquiry_Approved_360d


In [316]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression()
logistic_model.fit(X_train_model, y_train)

from sklearn.metrics import roc_curve

predict_train = logistic_model.predict_proba(X_train_model)
fpr,tpr,thresholds = roc_curve(y_train,predict_train[:,1])
print('KS-train:',max(tpr-fpr))

predict_test = logistic_model.predict_proba(X_test_model)
fpr,tpr,thresholds = roc_curve(y_test,predict_test[:,1])
print('KS-test:',max(tpr-fpr))

KS-train: 0.217460212079221
KS-test: 0.2112782480536594


In [317]:
predict_train = logistic_model.predict_proba(X_train_model)
prob_pred_train = predict_train[np.argsort(predict_train[:,1])][:,1]
prob_groups = pd.qcut(prob_pred_train, 10)

X_train_model['predict_proba'] = logistic_model.predict_proba(X_train_model)[:,1]
X_train_model['target'] = y_train
X_train_model['prob_groups'] = pd.qcut(X_train_model['predict_proba'], 10)

train = X_train_model[['predict_proba','target','prob_groups']].sort_values('prob_groups')
prob_data_train = pd.DataFrame(index=train['prob_groups'].unique())
prob_data_train['概率下限'] = prob_data_train.index.map(lambda x:x.right)
prob_data_train['概率上限'] = prob_data_train.index.map(lambda x:x.left)
prob_data_train['样本量'] = train['prob_groups'].groupby(prob_groups).count()
prob_data_train['正例数'] = train['target'].groupby(prob_groups).sum()
prob_data_train['正例比例'] = (prob_data_train['正例数']/prob_data_train['样本量']).apply(lambda x: format(x, '.2%')) 
prob_data_train['总体累积比例'] = (np.cumsum(prob_data_train['样本量'],axis=0)/np.sum(prob_data_train['样本量'],axis=0)).apply(lambda x: format(x, '.2%'))
prob_data_train['总体累积比例'] = np.array(prob_data_train['总体累积比例'])[::-1]
prob_data_train['正例累积比例'] = (np.cumsum(prob_data_train['正例数'],axis=0)/np.sum(prob_data_train['正例数'],axis=0)).apply(lambda x: format(x, '.2%'))
prob_data_train['正例累积比例'] = np.array(prob_data_train['正例累积比例'])[::-1]
prob_data_train['正例占总体累积比例'] = (np.cumsum(prob_data_train['正例数'],axis=0)/X_train_model.shape[0]).apply(lambda x: format(x, '.2%'))
prob_data_train

,概率下限,概率上限,样本量,正例数,正例比例,总体累积比例,正例累积比例,正例占总体累积比例
"(-0.000611, 0.074]",0.0740,-0.000611,1105,69,6.24%,100.00%,100.00%,0.62%
"(0.074, 0.0883]",0.0883,0.074000,1104,88,7.97%,89.99%,78.63%,1.42%
"(0.0883, 0.0997]",0.0997,0.088300,1104,84,7.61%,79.99%,64.71%,2.18%
"(0.0997, 0.11]",0.1100,0.099700,1104,104,9.42%,70.00%,52.06%,3.12%
"(0.11, 0.121]",0.1210,0.110000,1104,135,12.23%,60.00%,42.60%,4.35%
"(0.121, 0.133]",0.1330,0.121000,1104,150,13.59%,50.00%,32.45%,5.71%
"(0.133, 0.148]",0.1480,0.133000,1104,140,12.68%,40.00%,23.33%,6.97%
"(0.148, 0.169]",0.1690,0.148000,1104,187,16.94%,30.00%,16.29%,8.67%
"(0.169, 0.204]",0.2040,0.169000,1104,206,18.66%,20.01%,10.62%,10.53%
"(0.204, 0.876]",0.8760,0.204000,1105,316,28.60%,10.01%,4.67%,13.39%


In [318]:
x_test = X_test_model

In [319]:
predict_test = logistic_model.predict_proba(x_test)[:,1]

x_test['predict_proba'] = logistic_model.predict_proba(x_test)[:,1]
x_test['target'] = y_test


# 将测试集概率值划入训练集分组
bin = [0,1]
for i in prob_groups.unique(): # 训练集分割点
    bin.append(i.left)
bin.sort()
bin.pop(1)
x_test['prob_groups'] = pd.cut(predict_test, bin)
prob_groups_test = pd.cut(predict_test, bin)

test = x_test[['predict_proba','target','prob_groups']].sort_values('prob_groups')

prob_data_test = pd.DataFrame(index=test['prob_groups'].unique())
prob_data_test['概率下限'] = prob_data_test.index.map(lambda x:x.right)
prob_data_test['概率上限'] = prob_data_test.index.map(lambda x:x.left)
prob_data_test['样本量'] = test[['target','prob_groups']].groupby('prob_groups').count()
prob_data_test['正例数'] = test[['target','prob_groups']].groupby('prob_groups').sum()
prob_data_test['正例比例'] = (prob_data_test['正例数']/prob_data_test['样本量']).apply(lambda x: format(x, '.2%')) 
prob_data_test['总体累积比例'] = (np.cumsum(prob_data_test['样本量'],axis=0)/np.sum(prob_data_test['样本量'],axis=0)).apply(lambda x: format(x, '.2%'))
prob_data_test['总体累积比例'] = np.array(prob_data_test['总体累积比例'])[::-1]
prob_data_test['正例累积比例'] = (np.cumsum(prob_data_test['正例数'],axis=0)/np.sum(prob_data_test['正例数'],axis=0)).apply(lambda x: format(x, '.2%'))
prob_data_test['正例累积比例'] = np.array(prob_data_test['正例累积比例'])[::-1]
prob_data_test['正例占总体累积比例'] =  (np.cumsum(prob_data_test['正例数'],axis=0)/x_test.shape[0]).apply(lambda x: format(x, '.2%'))
prob_data_test

,概率下限,概率上限,样本量,正例数,正例比例,总体累积比例,正例累积比例,正例占总体累积比例
"(-0.000611, 0.074]",0.0740,-0.000611,695,51,7.34%,100.00%,100.00%,0.82%
"(0.074, 0.0883]",0.0883,0.074000,629,52,8.27%,87.09%,75.84%,1.65%
"(0.0883, 0.0997]",0.0997,0.088300,637,63,9.89%,77.88%,63.75%,2.66%
"(0.0997, 0.11]",0.1100,0.099700,593,70,11.80%,69.13%,52.10%,3.79%
"(0.11, 0.121]",0.1210,0.110000,619,85,13.73%,59.86%,42.72%,5.15%
"(0.121, 0.133]",0.1330,0.121000,559,75,13.42%,50.89%,34.63%,6.35%
"(0.133, 0.148]",0.1480,0.133000,578,87,15.05%,40.96%,25.46%,7.75%
"(0.148, 0.169]",0.1690,0.148000,546,108,19.78%,31.45%,17.91%,9.48%
"(0.169, 0.204]",0.2040,0.169000,574,112,19.51%,21.23%,11.11%,11.28%
"(0.204, 1.0]",1.0000,0.204000,805,224,27.83%,11.15%,5.50%,14.87%


In [339]:
for i in X_test_model.columns:
    print(i)
print(logistic_model.coef_)
print(logistic_model.intercept_)

Index(['af_n_Inquiry_Rejected_180d', 'af_n_Query_LoanCheck_all',
       'af_max_Odue_date_gap', 'af_max_Repay_M1_90d',
       'af_max_Inquiry_date_gap', 'af_max_Repay_Normal_all',
       'af_max_Repay_Clear_360d', 'jyzx_max_Inquiry_Approved_all',
       'jyzx_max_Inquiry_Pass_360d', 'jyzx_max_Repay_Normal_360d',
       'jyzx_max_Inquiry_date_gap', 'var47', 'af_n_Inquiry_360d',
       'jyzx_n_Inquiry_10d', 'predict_proba', 'target', 'prob_groups'],
      dtype='object')
[[ 4.69308240e-02  5.13958659e-02  2.09593203e-03  6.10250117e-04
  -1.26392797e-05 -6.43524452e-06 -7.11571902e-05 -5.48217915e-06
  -5.48058515e-06  2.26561212e-06 -2.38186714e-05 -3.45255692e-03
  -1.76291849e-02  1.52938033e-01]]
[-1.96747341]


#### IV筛选

In [429]:
def WOE(df, col, target):
    '''
    df: 包含需要计算WOE的变量和目标变量
    col : 需要计算WOE、IV的变量，必须是分箱后的变量
    target : 目标变量,0/1表示好/坏
    return :返回woe和iv
    '''
    total = df.groupby([col])['target'].count()
    total = pd.DataFrame({'total' : total})
    bad = df.groupby([col])['target'].sum()
    bad = pd.DataFrame({'bad' : bad})
    regroup = total.merge(bad, left_index=True, right_index=True, how='left')
    regroup.reset_index(level=0, inplace=True)
    N = sum(regroup['total'])
    B = sum(regroup['bad'])
    regroup['good'] = regroup['total'] - regroup['bad']
    G = N - B
    regroup['bad_pcnt'] = regroup['bad'].map(lambda x : x*1.0/B)
    regroup['good_pcnt'] = regroup['good'].map(lambda x : x*1.0/G)
    regroup['WOE'] = regroup.apply(lambda x : np.log(x.good_pcnt *1.0 /x.bad_pcnt), axis=1)
    WOE_dict = regroup[[col,'WOE']].set_index(col).to_dict(orient='index')
    for k, v in WOE_dict.items():
        WOE_dict[k] = v['WOE']
    IV = regroup.apply(lambda x:(x.good_pcnt - x.bad_pcnt) * np.log(x.good_pcnt/x.bad_pcnt), axis=1)
    IV = sum(IV)
    return {'WOE' : WOE_dict, "IV" : IV}

In [430]:
WOE_dict = {}
IV_dict = {}

for var in Indt_core_train.columns:
    if var !="target":
        woe_iv = WOE(Indt_core_train, var, "target")
        WOE_dict[var] = woe_iv['WOE']
        IV_dict[var] = woe_iv['IV']

        
# 当变量一个分组中，没有响应的数量 = 0时， 此时的IVi为+∞。
IV_dict_sorted = sorted(IV_dict.items(), key=lambda x :x[1],reverse=False)
for i in range(len(IV_dict_sorted)):
    if (IV_dict_sorted[i][1] != 0) and (IV_dict_sorted[i][1] != float("inf")) and (IV_dict_sorted[i][1] > 0.02):
        print(IV_dict_sorted[i][0])

af_max_Repay_Normal_360d
var16
var21
af_max_Inquiry_Rejected_30d
var24
var25
var20
af_max_Repay_Normal_all
var22
var17
af_max_Repay_M1_90d
af_max_Repay_M1plus_90d
af_max_Repay_M1_all
af_max_Repay_M1plus_all
af_max_Repay_Odue_90d
af_sum_Inquiry_Pass_rate_90d
af_sum_Inquiry_Pass_rate_360d
af_sum_Inquiry_Pass_rate_180d
af_sum_Inquiry_Pass_rate_all


In [227]:
X_train_model_2 = X_train[['var44', 'var10', 'var55', 'var47', 'var46', 'var15', 'var22', 'var17',
       'var24', 'var11', 'var19', 'var18', 'var26', 'var23', 'var35', 'var20',
       'var16', 'var25', 'var21', 'var27', 'var6', 'var13', 'var57', 'var9',
    'var1', 'var12', 'af_n_Query_LoanCheck_all', 'var7',
       'jyzx_max_Inquiry_date_gap', 'af_min_Inquiry_date_gap',
    'af_n_Query_NoLicConFin_all', 'var53', 'jyzx_min_Inquiry_date_gap',
       'af_max_Inquiry_date_gap', 'var42', 'af_sum_Inquiry_all', 'var8',
    'var38', 'af_sum_Inquiry_360d', 'af_sum_Inquiry_Rejected_all',
       'af_mean_Period_all', 'af_sum_Type_Credit_all', 'af_sum_Inquiry_180d',
    'jyzx_n_Repay_Unclear_all', 'jyzx_sum_Type_PersonalCredit_all',
       'af_n_Query_P2P_all', 'jyzx_n_Type_Personal_all',
    'af_n_Inquiry_Rejected_30d', 'var56', 'af_sum_Type_Credit_90d', 
    'af_max_Inquiry_Rejected_30d','jyzx_n_Inquiry_Checking_10d','jyzx_n_Inquiry_Pass_rate_360d','jyzx_sum_Inquiry_Pass_rate_360d','var18','var19','var23','af_max_Period_360d','var26','jyzx_min_Period_360d','af_n_Repay_M1_180d','af_n_Repay_M1plus_180d','af_n_Repay_Odue_180d','af_n_Repay_M1_90d','af_n_Repay_M1plus_90d','var16','var21','af_max_Repay_M1_90d','af_max_Repay_M1plus_90d','af_sum_Repay_M1_90d','af_sum_Repay_M1plus_90d','af_n_Repay_M1_all','af_n_Repay_M1plus_all','af_n_Repay_Odue_all','af_max_Repay_M1_all','af_max_Repay_M1plus_all','af_sum_Repay_M1_all','af_sum_Repay_M1plus_all','af_n_Repay_M1_360d','af_n_Repay_M1plus_360d','af_n_Repay_Odue_90d','var25','var20','af_n_Repay_Odue_360d','af_max_Repay_Odue_90d','af_sum_Repay_Odue_90d','jyzx_max_Period_360d','var38','var53']]

X_test_model_2 = X_test[['var44', 'var10', 'var55', 'var47', 'var46', 'var15', 'var22', 'var17',
       'var24', 'var11', 'var19', 'var18', 'var26', 'var23', 'var35', 'var20',
       'var16', 'var25', 'var21', 'var27', 'var6', 'var13', 'var57', 'var9',
    'var1', 'var12', 'af_n_Query_LoanCheck_all', 'var7',
       'jyzx_max_Inquiry_date_gap', 'af_min_Inquiry_date_gap',
    'af_n_Query_NoLicConFin_all', 'var53', 'jyzx_min_Inquiry_date_gap',
       'af_max_Inquiry_date_gap', 'var42', 'af_sum_Inquiry_all', 'var8',
    'var38', 'af_sum_Inquiry_360d', 'af_sum_Inquiry_Rejected_all',
       'af_mean_Period_all', 'af_sum_Type_Credit_all', 'af_sum_Inquiry_180d',
    'jyzx_n_Repay_Unclear_all', 'jyzx_sum_Type_PersonalCredit_all',
       'af_n_Query_P2P_all', 'jyzx_n_Type_Personal_all',
    'af_n_Inquiry_Rejected_30d', 'var56', 'af_sum_Type_Credit_90d', 
    'af_max_Inquiry_Rejected_30d','jyzx_n_Inquiry_Checking_10d','jyzx_n_Inquiry_Pass_rate_360d','jyzx_sum_Inquiry_Pass_rate_360d','var18','var19','var23','af_max_Period_360d','var26','jyzx_min_Period_360d','af_n_Repay_M1_180d','af_n_Repay_M1plus_180d','af_n_Repay_Odue_180d','af_n_Repay_M1_90d','af_n_Repay_M1plus_90d','var16','var21','af_max_Repay_M1_90d','af_max_Repay_M1plus_90d','af_sum_Repay_M1_90d','af_sum_Repay_M1plus_90d','af_n_Repay_M1_all','af_n_Repay_M1plus_all','af_n_Repay_Odue_all','af_max_Repay_M1_all','af_max_Repay_M1plus_all','af_sum_Repay_M1_all','af_sum_Repay_M1plus_all','af_n_Repay_M1_360d','af_n_Repay_M1plus_360d','af_n_Repay_Odue_90d','var25','var20','af_n_Repay_Odue_360d','af_max_Repay_Odue_90d','af_sum_Repay_Odue_90d','jyzx_max_Period_360d','var38','var53']]

In [229]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
forest.fit(X_train_model_2, y_train)

importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

feat_labels = X_train_model_2.columns
for f in range(X_train_model_2.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) var44                          0.045963
 2) var55                          0.044849
 3) var10                          0.044467
 4) var47                          0.043818
 5) var46                          0.033172
 6) var22                          0.027260
 7) var15                          0.026788
 8) var17                          0.025419
 9) var24                          0.023468
10) jyzx_max_Inquiry_date_gap      0.019969
11) af_max_Inquiry_date_gap        0.019614
12) jyzx_min_Inquiry_date_gap      0.019385
13) jyzx_sum_Type_PersonalCredit_all 0.019026
14) var11                          0.018793
15) af_n_Query_LoanCheck_all       0.018256
16) jyzx_n_Type_Personal_all       0.017282
17) af_min_Inquiry_date_gap        0.017248
18) af_sum_Inquiry_360d            0.017141
19) var35                          0.016940
20) af_sum_Inquiry_180d            0.016779
21) af_sum_Type_Credit_all         0.016619
22) af_sum_Inquiry_Rejected_all    0.016604
23) af_sum_Inquiry_all        